In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import seaborn as sns
import scipy

In [2]:
#import and check customer data
path = '/Users/kimballwightman/Documents/CareerFoundry/Instacart Basket Analysis'

In [3]:
df_customer = pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'))

In [7]:
df_customer.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [5]:
df_customer.shape

(206209, 10)

In [4]:
#wrangle data
df_customer.rename(columns = {'Surnam':'Last Name'}, inplace = True)

In [5]:
df_customer.rename(columns = {'n_dependants':'num_dependants'}, inplace = True)

I'm choosing not to drop any columns yet, I'm still not sure what columns I'll need for the final project. I made that mistake in Achievement 1 and I'm not going to do it again.

In [10]:
df_customer.head()

user_id First Name Last Name  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   num_dependants fam_status  income  
0               3    married  165665  
1               0     single   59285  
2               2    married   99568  
3               0     single   42049  
4               1    married   40374

In [11]:
#data quality/consistency checks
for col in df_customer.columns.tolist():
    weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(df_customer[weird]) > 0:
        print (col)

First Name


In [6]:
#fix mixed-type column
df_customer['First Name'] = df_customer['First Name'].astype('str')

In [22]:
df_customer.dtypes

user_id            int64
First Name        object
Last Name         object
Gender            object
STATE             object
Age                int64
date_joined       object
num_dependants     int64
fam_status        object
income             int64
dtype: object

In [24]:
#missing values
df_customer.isnull().sum()

user_id           0
First Name        0
Last Name         0
Gender            0
STATE             0
Age               0
date_joined       0
num_dependants    0
fam_status        0
income            0
dtype: int64

No missing data.

In [25]:
#duplicates
df_dups = df_customer[df_customer.duplicated()]

In [26]:
df_dups

Empty DataFrame
Columns: [user_id, First Name, Last Name, Gender, STATE, Age, date_joined, num_dependants, fam_status, income]
Index: []

No duplicates.

In [7]:
#merge your data sets (and import your orders/products set)
ords_prods = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_group.pkl'))

In [15]:
ords_prods.head()

order_id  user_id  user_order_num  order_day_of_week  order_hour_of_day  \
0   2539329        1               1                  2                  8   
1   2398795        1               2                  3                  7   
2    473747        1               3                  3                 12   
3   2254736        1               4                  4                  7   
4    431534        1               5                  4                 15   

   days_since_prior_order  first_order  product_id  add_to_cart_order  \
0                     NaN         True         196                  1   
1                    15.0        False         196                  1   
2                    21.0        False         196                  1   
3                    29.0        False         196                  1   
4                    28.0        False         196                  1   

   reordered  ...    price_range_loc     busiest_day     busiest_days  \
0          0  ...  Mid-Range Product  Regularly Busy   Regularly Busy   
1          1  ...  Mid-Range Product  Regularly Busy  Least Busy Days   
2          1  ...  Mid-Range Product  Regularly Busy  Least Busy Days   
3          1  ...  Mid-Range Product      Least Busy  Least Busy Days   
4          1  ...  Mid-Range Product      Least Busy  Least Busy Days   

   busiest_period_of_day max_order  loyalty_flag avg_spend spending_flag  \
0         Average Orders        10  New Customer  6.367797   Low Spender   
1         Average Orders        10  New Customer  6.367797   Low Spender   
2            Most Orders        10  New Customer  6.367797   Low Spender   
3         Average Orders        10  New Customer  6.367797   Low Spender   
4            Most Orders        10  New Customer  6.367797   Low Spender   

  median_dspo         frequency_flag  
0        20.5  Non-Frequent Customer  
1        20.5  Non-Frequent Customer  
2        20.5  Non-Frequent Customer  
3        20.5  Non-Frequent Customer  
4        20.5  Non-Frequent Customer  

[5 rows x 25 columns]

In [30]:
ords_prods['user_id'].dtype

dtype('int64')

In [8]:
ords_prods = ords_prods.drop(columns = ['_merge'])

In [9]:
customers_merged = ords_prods.merge(df_customer, on = ['user_id'], indicator = True)

In [10]:
customers_merged.head()

order_id  user_id  user_order_num  order_day_of_week  order_hour_of_day  \
0   2539329        1               1                  2                  8   
1   2398795        1               2                  3                  7   
2    473747        1               3                  3                 12   
3   2254736        1               4                  4                  7   
4    431534        1               5                  4                 15   

   days_since_prior_order  first_order  product_id  add_to_cart_order  \
0                     NaN         True         196                  1   
1                    15.0        False         196                  1   
2                    21.0        False         196                  1   
3                    29.0        False         196                  1   
4                    28.0        False         196                  1   

   reordered  ... First Name  Last Name  Gender    STATE Age date_joined  \
0          0  ...      Linda     Nguyen  Female  Alabama  31   2/17/2019   
1          1  ...      Linda     Nguyen  Female  Alabama  31   2/17/2019   
2          1  ...      Linda     Nguyen  Female  Alabama  31   2/17/2019   
3          1  ...      Linda     Nguyen  Female  Alabama  31   2/17/2019   
4          1  ...      Linda     Nguyen  Female  Alabama  31   2/17/2019   

  num_dependants fam_status  income _merge  
0              3    married   40423   both  
1              3    married   40423   both  
2              3    married   40423   both  
3              3    married   40423   both  
4              3    married   40423   both  

[5 rows x 34 columns]

In [11]:
customers_merged['_merge'].value_counts(dropna = False)

both          32089823
left_only            0
right_only           0
Name: _merge, dtype: int64

In [12]:
#export the file in pickle format
customers_merged.to_pickle(os.path.join(path,'02 Data','Prepared Data','customers_merged.pkl'))